# Generate helmet training set for FairMOT

In [1]:
import os, sys
import random
import numpy as np
import pandas as pd
import torch
import cv2
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [2]:
train = pd.read_csv('/kaggle/input/nfl-health-and-safety-helmet-assignment/train_labels.csv')

# Get the unique helmets (i.e. players) in this dataset.
unique_video_labels = train.groupby(['video', 'label']).count().index.values
dict_classid = {video_label: i + 1 for i, video_label in enumerate(unique_video_labels)}

# Assign each unique helmet/player a unique id, `classid`.
train['classid'] = train.set_index(['video', 'label']).index.map(dict_classid)

# Replace player number (e.g. H22, V13, etc.) with `classid` as labels
train.drop('label', axis=1, inplace=True)
train.rename(columns={'classid': 'label'}, inplace=True)
train['label'] = train['label'].astype(str)

In [3]:
dir_root = '/kaggle/working/NFL'
dir_img = os.path.join(dir_root, 'images')
dir_label = os.path.join(dir_root, 'labels_with_ids')

os.makedirs(dir_img, exist_ok=True)
os.makedirs(dir_label, exist_ok=True)

In [4]:
unique_videos = train['video'].unique()

In [5]:
for fn_video in tqdm(unique_videos, total=len(unique_videos)):

    video_id = fn_video.split('.')[0]
    dir_img_video = os.path.join(dir_img, 'train', video_id, 'img1')
    dir_label_video = os.path.join(dir_label, 'train', video_id, 'img1')
    os.makedirs(dir_img_video, exist_ok=True)
    os.makedirs(dir_label_video, exist_ok=True)

    pth_video = os.path.join('/kaggle/input/nfl-health-and-safety-helmet-assignment/train', fn_video)
    cap = cv2.VideoCapture(pth_video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    df_video = train[train['video'] == fn_video]

    for frame, df_frame in tqdm(df_video.groupby('frame'), total=len(df_video.groupby('frame'))):
        read_ok, img = cap.read()
        if not read_ok:
            break

        cv2.imwrite(os.path.join(dir_img_video, f'{frame:06d}.jpg'), img)

        df_frame['box_centre_x'] = df_frame['left'] + .5 * df_frame['width']
        df_frame['box_centre_y'] = df_frame['top'] + .5 * df_frame['height']
        df_frame['box_width'] = df_frame['width'].copy()
        df_frame['box_height'] = df_frame['height'].copy()
        df_frame['box_centre_x'] = (df_frame['box_centre_x'] / width).round(6)
        df_frame['box_centre_y'] = (df_frame['box_centre_y'] / height).round(6)
        df_frame['box_width'] = (df_frame['box_width'] / width).round(6)
        df_frame['box_height'] = (df_frame['box_height'] / height).round(6)
        df_frame['dummy'] = 0

        df_txt = df_frame[['dummy', 'label', 'box_centre_x', 'box_centre_y', 'box_width', 'box_height']]
        pth_txt = os.path.join(dir_label_video, f'{frame:06d}.txt')
        df_txt.to_csv(pth_txt, sep=' ', header=False, index=False)        

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/472 [00:00<?, ?it/s]

  0%|          | 0/471 [00:00<?, ?it/s]

  0%|          | 0/615 [00:00<?, ?it/s]

  0%|          | 0/617 [00:00<?, ?it/s]

...

  0%|          | 0/350 [00:00<?, ?it/s]

  0%|          | 0/549 [00:00<?, ?it/s]

  0%|          | 0/548 [00:00<?, ?it/s]

  0%|          | 0/473 [00:00<?, ?it/s]

  0%|          | 0/473 [00:00<?, ?it/s]